In [9]:
!ls /keyakizaka_mining/model/facenet

20180402-114759  20180408-102900


In [10]:
# path

save_dir = '/keyakizaka_mining/data/images'
save_feature_dir = '/keyakizaka_mining/data/feature/image'
FACE_MEDEL_PATH = '/keyakizaka_mining/model/facenet/20180408-102900/20180408-102900.pb'

In [3]:
import os, sys
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import cloudpickle
def save_as_pkl( obj, fpath ):
    fwb = open( fpath, 'wb')
    tmp = fwb.write(cloudpickle.dumps(obj))
    return

In [6]:
# https://tech.unifa-e.com/entry/2018/09/20/183742
# > git clone https://github.com/davidsandberg/facenet.git

# add path for facenet model
sys.path.append('/keyakizaka_mining/tools')
from facenet.src import facenet

class FaceEmbedding(object):

    def __init__(self, model_path):
        facenet.load_model(model_path)
        
        self.input_image_size = 160
        self.sess = tf.Session()
        self.images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        self.embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        self.phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        self.embedding_size = self.embeddings.get_shape()[1]
        
    def __del__(self):
        self.sess.close()
        
    def load_image(self, image_path, width, height, mode):
        image = Image.open(image_path)
        image = image.resize([width, height], Image.BILINEAR)
        return np.array(image.convert(mode))
        
    def face_embeddings(self, image_path):
        image = self.load_image(image_path, self.input_image_size, self.input_image_size, 'RGB')
        prewhitened = facenet.prewhiten(image)
        prewhitened = prewhitened.reshape(-1, prewhitened.shape[0], prewhitened.shape[1], prewhitened.shape[2])
        feed_dict = { self.images_placeholder: prewhitened, self.phase_train_placeholder: False }
        embeddings = self.sess.run(self.embeddings, feed_dict=feed_dict)
        return embeddings

In [7]:
# read csv

df = pd.read_csv( '/keyakizaka_mining/data/blog/keyakizaka46_blog_20190217.csv' )
print(df.shape); display( df.head() )

(16754, 6)


,author,datetime,title,text,images,url
0,齋藤冬優花,2015/11/1400:02,＼はじめまして︎︎❤︎︎1/︎︎,\n お見立て会前日にして始まりましたっ!欅坂46ブログ(*ﾟ▽ﾟ)ﾉ嬉しくて嬉しくて、、、...,http://cdn.keyakizaka46.com/files/14/diary/k46...,http://www.keyakizaka46.com/s/k46o/diary/detai...
1,菅井友香,2015/11/1323:54,菅井友香です(^ ^),\n初めまして(^ ^)!この度欅坂46として活動させて頂けることになりました、東京都出身1...,http://cdn.keyakizaka46.com/files/14/diary/k46...,http://www.keyakizaka46.com/s/k46o/diary/detai...
2,尾関梨香,2015/11/1323:52,じゃじゃじゃーーん①,\nはじめまして(^ν^)神奈川県出身高校3年生18歳の尾関梨香です。いよいよ本日からブログ...,http://cdn.keyakizaka46.com/files/14/diary/k46...,http://www.keyakizaka46.com/s/k46o/diary/detai...
3,小林由依,2015/11/1323:51,はじまりました☆彡,\n「皆」という漢字の下の部分が 日 ではなく、白ということを最近知りました。小林由依です♪...,NaN,http://www.keyakizaka46.com/s/k46o/diary/detai...
4,小池美波,2015/11/1323:49,みなさん初めまして、小池美波です♪,\nみなさん初めまして!高校二年生、兵庫県出身の小池美波です!ついに明日は欅坂46の初イベン...,http://cdn.keyakizaka46.com/files/14/diary/k46...,http://www.keyakizaka46.com/s/k46o/diary/detai...


In [6]:
# save facenet features

# load model

face_embedding = FaceEmbedding(FACE_MEDEL_PATH)

dirs = os.listdir( save_dir )
for author in dirs:
    if os.path.exists( '%s/facenet/X_facenet_%s.pkl' % (save_feature_dir, author) ): continue
    print( author, "Extract features..." )
    X = []
    y = []
    image_list  = []
    images = os.listdir( '%s/%s'% (save_dir,author) )
    for idx, imname in enumerate( images ):
        if idx % 100 == 0: print( idx )
        img_path = '%s/%s/%s'% (save_dir,author,imname)
        feature = face_embedding.face_embeddings( img_path )[0]
        X.append( feature.flatten() )
        y.append( author )
        image_list.append( img_path )
    
    # save feature
    X = np.array(X)
    save_as_pkl( X, '%s/facenet/X_facenet_%s.pkl' % (save_feature_dir, author) )
    save_as_pkl( y, '%s/facenet/y_facenet_%s.pkl' % (save_feature_dir, author) )
    save_as_pkl( image_list, '%s/facenet/imagelist_facenet_%s.pkl' % (save_feature_dir, author) )

Model filename: ../model/20180402-114759/20180402-114759.pb
